In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import json, os, time

import pyautogui
from PIL import Image
import cv2
import numpy as np
from dotenv import load_dotenv
import openai

from prompts import *
from GUIUtils import *
from GUIToolbox import LightroomGUIToolBox
from Chat import AgentClient
from ChatClaude import ClaudeAgentClient
from cognitive_architecture import adjustment_routing_woKI

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key

## First Step: Set the window, set clibration points for automated GUI operations
Set up window and calibration points for automated GUI control
This step needs to be done once and the configuration will be saved for future use
Steps:
1. Position and size the Lightroom window appropriately 
2. Follow prompts to calibrate control points for automation
3. Configuration will be saved to a file for reuse
4. Remeber to give terminal right to control your computer

In [11]:
# GUI_config_file = "./GUI_config_half.json"
GUI_config_file = "./GUI_config_1.json"
# GUI_config_file = "./GUI_config.json"
update_setting_image = 'UpdateSetting.png'

if os.path.exists(GUI_config_file):
    print(f"Configuration file '{GUI_config_file}' found. Loading...")
    with open(GUI_config_file, "r") as config_file:
        GUI_config = json.load(config_file)
else:
    button_image = os.path.abspath("./button.png")
    color_button_image = os.path.abspath("./color-button.png")
    HDPI = 2
    background_color = [26, 26, 26]
    pannel_and_histogram_area = set_calibration_points()
    slider_positions = record_slider_positions(button_image)
    color_slider_positions = record_color_slider_positions(color_button_image)
    GUI_config = create_toolbox_config_template(button_image, color_button_image, HDPI, background_color, pannel_and_histogram_area, slider_positions, color_slider_positions)
    with open(GUI_config_file, "w") as config_file:
        json.dump(GUI_config, config_file, indent=4)

Configuration file './GUI_config_1.json' found. Loading...


### Run test to control the GUI Lightroom

In [ ]:
skip_legacy_setting_button(update_setting_image, GUI_config)
set_slider_positions(test_parameters_2, GUI_config)
set_slider_positions(test_parameters_3, GUI_config)
set_slider_positions(test_parameters_1, GUI_config)

set_color_slider_positions(color_test_parameters_2, GUI_config)
set_color_slider_positions(color_test_parameters_3, GUI_config)
set_color_slider_positions(color_test_parameters_1, GUI_config)

## Test single image

In [ ]:
skip_legacy_setting_button(update_setting_image, GUI_config)
set_slider_positions(test_parameters_1, GUI_config)
set_color_slider_positions(color_test_parameters_1, GUI_config)

In [15]:
exp_index = 31

output_dir_path = f"./test-output/P./test-output/PhotoArtAgent_V.Jan5_woKI_artistic_emphasis/test_image_withcolor_{exp_index}"
GUIToolbox = LightroomGUIToolBox(GUI_config, output_dir_path, image_name=f"test_image{exp_index}", clip_history_messages=True)
global_style = photo_editing_global_styles["artistic_emphasis"]

User messgae file path: /Users/haoyu/Research/Retouching/AgentRetouching-main 2/test-output/P./test-output/PhotoArtAgent_V.Jan5_woKI_artistic_emphasis/test_image_withcolor_31/user_messages.json


### Claude

In [16]:
anthropic_api_key = ""

from ChatClaude import ClaudeAgentClient

chat_client = ClaudeAgentClient(api_key=anthropic_api_key, 
                                model='claude-3-sonnet-20240229', 
                                toolbox_instance=GUIToolbox, 
                                debug=False)


If you need to view the messages. Now select the output messages.json file on the server.

In [17]:
adjustment_routing_woKI(chat_client, 
                        GUIToolbox, 
                        global_style=global_style, 
                        retry=5)

`func_return_responses`, response: 

1. **Image Content Description:** The image captures a majestic mountain landscape with snow-capped peaks towering over a lush, green valley. A winding river meanders through the meadow, surrounded by dense forests. Blooming trees in the foreground add a touch of vibrant color.

2. **General Requirements for Retouching Approaches:**
*Light Adjustments:* Exposure, contrast, highlights, shadows, blacks, and whites adjustments will vary based on the desired mood and effect.
*Color Adjustments:* The overall color temperature, tint, vibrance, and saturation will be tailored to match the emotional tone of each retouching approach.
*Individual Color Adjustments:* Specific hues like green, blue, and white may be adjusted in saturation and luminance to enhance or subdue their presence.

3. **Retouching Approaches:**

*Approach 1 (Balanced/Subtle):* Maintain natural lighting with a slight increase in exposure and contrast. Preserve vibrant greens and blues wh

### ChatGPT

In [18]:
openai_api_key = ""

from Chat import AgentClient

chat_client = AgentClient(api_key=openai_api_key, 
                          model='gpt-4o-2024-11-20', 
                          toolbox_instance=GUIToolbox, 
                          debug=False)


In [19]:
adjustment_routing_woKI(chat_client, 
                        GUIToolbox, 
                        global_style=global_style, 
                        retry=5)

`func_return_responses`, response: 

### Image Description
The image presents a breathtaking mountain valley scene with snow-capped peaks, expansive green forests, and a meandering stream. The foreground features vibrant green foliage, while the distant view showcases glacial remnants under a partly cloudy sky.

### Retouching Proposals

**Approach 1: Balanced/Subtle Adjustments**
- **Light:** Increase exposure (+0.2), raise shadows (+20), reduce highlights (-15), and slightly deepen blacks (-5) for detail balance.
- **Color:** Set temperature to slightly cool (5500K), enhance vibrance (+15) for clarity.
- **Individual Colors:** Boost green saturation (+15) and blue luminance (+10), ensuring colors remain authentic and harmonious.

**Approach 2: Contrasting/Moody Adjustments**
- **Light:** Boost contrast significantly (+30), deepen shadows (-20), and reduce highlights (-30) to emphasize depth.
- **Color:** Shift temperature cooler (5000K) and slightly increase saturation (+10) for inte

## Run batch images

In [ ]:
processing_image_number = 114
for i in range(processing_image_number):
    exp_index = i
    skip_legacy_setting_button(update_setting_image, GUI_config)
    set_slider_positions(test_parameters_1, GUI_config)
    set_color_slider_positions(color_test_parameters_1, GUI_config)
    output_dir_path = f"./test-output/PhotoArtAgent_V.Jan5_woKI/test_image_withcolor_{exp_index}"
    GUIToolbox = LightroomGUIToolBox(GUI_config, output_dir_path, image_name=f"test_image{exp_index}", clip_history_messages=True)
    chat_client = AgentClient(api_key=api_key, toolbox_instance=GUIToolbox, debug=False)
    global_style = photo_editing_global_styles["artistic_emphasis"]

    adjustment_routing_woKI(chat_client, GUIToolbox, global_style=global_style, retry=5)

    time.sleep(2)
    pyautogui.press('right')
    time.sleep(1)